In [1]:
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
client = OpenAI()

In [2]:
import os
import sys
import base64
import json
import pandas as pd

In [3]:
root_directory = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(os.path.join(root_directory,"src"))

from backend_functions import gpt_ocr, timetabledata

In [6]:
import importlib
importlib.reload(gpt_ocr)
importlib.reload(timetabledata)

<module 'backend_functions.timetabledata' from 'c:\\Users\\kkoaz\\Documents\\projects\\product\\timetable-detect\\timetable-detect\\src\\backend_functions\\timetabledata.py'>

In [10]:
prompt_system_info = """
あなたは日本語に長けていて、また日本のアイドルシーンに詳しい専門家です。
与えられる画像は日本のアイドルのフェスについてのタイムテーブルです。
そこから情報を読み取り、
・ライブの名前
・会場の名前
・日付
・ステージ数
・各ステージの名前
を下記のようなJSON形式で出力してください。

{
    "ライブ名":"TOKYO GIRLS GIRLS"
    ,"会場名":"ZeppDiverCity"
    ,"日付":"2024-08-03"
    ,"ステージ数":3
    ,"ステージ名":["スマイルガーデン","SKY STAGE","ガンダム広場"]
}
"""

In [9]:
doc_folder = os.getcwd()+"/../../data/"
image_path = doc_folder + "timetable_sample/20240503_JCIF-LIVE.jpeg"
response = gpt_ocr.getocr(image_path, "このタイムテーブルの情報を教えて", prompt_system_info)
json.loads(response.choices[0].message.content)

{'ライブ名': 'JAPAN CENTRAL IDOL FESTIVAL 2024',
 '会場名': 'AICHI SKY EXPO',
 '日付': '2024-05-03',
 'ステージ数': 5,
 'ステージ名': ['CENTRAL STAGE',
  'DREAM STAGE',
  'SKY STAGE',
  'OCEAN STAGE',
  'CORAL STAGE']}

In [15]:
image_path = doc_folder + "timetable_sample/20240616_RADJAM.jpg"
response = gpt_ocr.getocr(image_path, "このタイムテーブルの情報を教えて", prompt_system_info)
json.loads(response.choices[0].message.content)

{'ライブ名': 'RAD JAM',
 '会場名': 'AICHI SKY EXPO',
 '日付': '2024-06-16',
 'ステージ数': 3,
 'ステージ名': ['SUPER!GMG STAGE (Bホール)',
  'NAGOYA ReNY Limited (Cホール)',
  'FaveConnect STAGE (Dホール野外)']}

In [11]:
prompt_system_timetable = """
あなたは日本語に長けていて、また日本のアイドルシーンに詳しい専門家です。
与えられる画像は日本のアイドルのフェスについてのタイムテーブルです。
画像の中から、ユーザーから指定されたステージ名についての情報のみを抽出し、
そのタイムテーブルの情報（出演するアイドルの一覧とその出演時間）を
JSON形式で出力してください。

#出演時間について
属性として「開始時間(from)」「終了時間(to)」を持ちます。
ライブステージは一つのアイドルグループが複数回出演する場合がありますが、
複数回のステージはそれぞれ異なるステージとして出力してください。
出演時間は、グループ名の近くに記載されている場合が多いので、それを採用してください。
もし明確な出演時間が記載されていない場合には、
グループ名を囲む四角形の辺の位置を、
タイムテーブル上の時刻の目盛りから読み取ることによって時間を特定してください。

##終演後特典会について
「終演後物販」「終演後特典会」といった演目が存在している場合がありますが、
これはライブステージではないので無視をしてください。

#出力形式
以下の様なJSON形式で出力してください。
{
    "ステージ名":"HOTステージ"
    ,"タイムテーブル":
    [
        {
            "グループ名":"アイドルグループA"
            ,"ライブステージ":
            {
                "from":"10:00"
                ,"to":"10:20"
            },
        },
        {
            "グループ名":"架空のアイドル"
            ,"ライブステージ":
            {
                "from":"10:25"
                ,"to":"10:45"
            },
        }
    ]
}
"""

In [12]:
prompt_user = "ステージ名：CENTRAL STAGE"
doc_folder = os.getcwd()+"/../../data/"
image_path = doc_folder + "timetable_sample/20240503_JCIF-LIVE.jpeg"
response = gpt_ocr.getocr(image_path, prompt_user, prompt_system_timetable)
json.loads(response.choices[0].message.content)

{'ステージ名': 'CENTRAL STAGE',
 'タイムテーブル': [{'グループ名': 'THE ENCORE',
   'ライブステージ': {'from': '9:50', 'to': '10:10'}},
  {'グループ名': 'アンター・ビーナスティー', 'ライブステージ': {'from': '10:15', 'to': '10:35'}},
  {'グループ名': 'dela', 'ライブステージ': {'from': '10:40', 'to': '11:10'}},
  {'グループ名': "瑠璃色♪drop'tune♪", 'ライブステージ': {'from': '11:15', 'to': '11:35'}},
  {'グループ名': 'IMyME!MINE', 'ライブステージ': {'from': '11:40', 'to': '12:00'}},
  {'グループ名': '仮面女子', 'ライブステージ': {'from': '12:05', 'to': '12:25'}},
  {'グループ名': '虹の架け橋少女', 'ライブステージ': {'from': '12:30', 'to': '12:50'}},
  {'グループ名': 'なぉナ王の房ィ後', 'ライブステージ': {'from': '12:55', 'to': '13:15'}},
  {'グループ名': 'ワンダーウィード 天', 'ライブステージ': {'from': '13:20', 'to': '13:40'}},
  {'グループ名': '平我ゼセーシヨン', 'ライブステージ': {'from': '13:45', 'to': '14:05'}},
  {'グループ名': '#2i2', 'ライブステージ': {'from': '14:10', 'to': '14:30'}},
  {'グループ名': "MyDearDarlin'", 'ライブステージ': {'from': '14:35', 'to': '14:55'}},
  {'グループ名': 'SAISON', 'ライブステージ': {'from': '15:00', 'to': '15:20'}},
  {'グループ名': 'アイドルブレジン', 'ライブステージ': {'from': 

In [13]:
prompt_user = "ステージ名：OCEAN STAGE"
doc_folder = os.getcwd()+"/../../data/"
image_path = doc_folder + "timetable_sample/20240503_JCIF-LIVE.jpeg"
response = gpt_ocr.getocr(image_path, prompt_user, prompt_system_timetable)
json.loads(response.choices[0].message.content)

{'ステージ名': 'OCEAN STAGE',
 'タイムテーブル': [{'グループ名': 'テンシメシ໒3', 'ライブステージ': {'from': '10:00', 'to': '10:20'}},
  {'グループ名': '≒JOY', 'ライブステージ': {'from': '10:20', 'to': '10:40'}},
  {'グループ名': 'アポストロフィ', 'ライブステージ': {'from': '10:40', 'to': '11:00'}},
  {'グループ名': 'OS☆K', 'ライブステージ': {'from': '11:00', 'to': '11:20'}},
  {'グループ名': 'Symdolick', 'ライブステージ': {'from': '11:20', 'to': '11:40'}},
  {'グループ名': 'HAPPY⤴️⤴️', 'ライブステージ': {'from': '11:40', 'to': '12:00'}},
  {'グループ名': 'あっとーふぇむ', 'ライブステージ': {'from': '12:00', 'to': '12:20'}},
  {'グループ名': 'かかぴるファミリーにおまかせ', 'ライブステージ': {'from': '12:20', 'to': '12:40'}},
  {'グループ名': 'HIGH SPIRITS', 'ライブステージ': {'from': '12:40', 'to': '13:00'}},
  {'グループ名': '戦国アニマル極楽浄土', 'ライブステージ': {'from': '13:00', 'to': '13:20'}},
  {'グループ名': 'Chuz-', 'ライブステージ': {'from': '13:20', 'to': '13:40'}},
  {'グループ名': 'ミスティア！', 'ライブステージ': {'from': '13:40', 'to': '14:00'}},
  {'グループ名': 'えぬえむとぅけ', 'ライブステージ': {'from': '14:00', 'to': '14:20'}},
  {'グループ名': 'ひめもすオーケストラ', 'ライブステージ': {'from': '14:20', 't

In [15]:
import importlib
importlib.reload(gpt_ocr)
importlib.reload(timetabledata)

<module 'backend_functions.timetabledata' from 'c:\\Users\\kkoaz\\Documents\\projects\\product\\timetable-detect\\timetable-detect\\src\\backend_functions\\timetabledata.py'>

In [6]:
doc_folder = os.getcwd()+"/../../data/projects/"
image_path = doc_folder + "202507_SWEETSUMMER/event_1/ライブ特典会/raw_cropped.png"
# response = gpt_ocr.getocr_fes_timetable_functioncalling(image_path)
response = gpt_ocr.getocr_fes_stagelist_structured(image_path,3)
print(response)

{'ステージ名': ['WWWX', 'Veats', 'VIDENT'], '命名規則': '特になし'}


In [16]:
doc_folder = os.getcwd()+"/../../data/projects/"
image_path = doc_folder + "TGG20250105/event_1/ライブ特典会/stage_1.png"
# response = gpt_ocr.getocr_fes_timetable_functioncalling(image_path)
response = gpt_ocr.getocr_fes_withtokutenkai_timetable_functioncalling(image_path)
print(response)

1
{'タイムテーブル': [{'グループ名': '天上天使', 'ライブステージ': {'from': '09:20', 'to': '09:40'}, '特典会': [{'from': '09:50', 'to': '11:00', 'ブース': 'A'}]}, {'グループ名': 'コングラッチュエット！', 'ライブステージ': {'from': '09:40', 'to': '10:00'}, '特典会': [{'from': '10:10', 'to': '11:20', 'ブース': 'B'}]}, {'グループ名': 'Krays', 'ライブステージ': {'from': '10:00', 'to': '10:20'}, '特典会': [{'from': '10:30', 'to': '11:40', 'ブース': 'C'}]}, {'グループ名': 'メリーパレード', 'ライブステージ': {'from': '10:20', 'to': '10:40'}, '特典会': [{'from': '10:50', 'to': '12:00', 'ブース': 'D'}]}, {'グループ名': 'シェルム', 'ライブステージ': {'from': '10:40', 'to': '11:00'}, '特典会': [{'from': '11:10', 'to': '12:20', 'ブース': 'A'}]}, {'グループ名': '青群リグレット', 'ライブステージ': {'from': '11:00', 'to': '11:20'}, '特典会': [{'from': '11:30', 'to': '12:40', 'ブース': 'B'}]}, {'グループ名': 'みらウィ！', 'ライブステージ': {'from': '11:20', 'to': '11:40'}, '特典会': [{'from': '11:50', 'to': '13:00', 'ブース': 'C'}]}, {'グループ名': 'ROOKEY♡ROOKEYS', 'ライブステージ': {'from': '11:45', 'to': '12:05'}, '特典会': [{'from': '12:15', 'to': '13:25', 'ブース': 'A'}]}, {'グループ名'

In [24]:
doc_folder = os.getcwd()+"/../../data/projects/"
image_path = doc_folder + "atjam2024/event_1/ライブ/raw_cropped.png"
response = gpt_ocr.getocr_fes_stagelist(image_path, 6)
print(response)

(['ストロベリーステージ', 'キウイステージ', 'ブルーベリーステージ', 'ピーチステージ', 'オレンジステージ', 'パイナップルステージ'], '特になし')


In [25]:
doc_folder = os.getcwd()+"/../../data/projects/"
image_path = doc_folder + "atjam2024/event_1/特典会/raw_cropped.png"
response = gpt_ocr.getocr_fes_stagelist(image_path, 17)
print(response)

(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17'], '数字')
